In [1]:
import scanpy as sc
import os
import numpy as np
from tqdm.notebook import tqdm
import glob
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from tensorflow import keras

### Put in the path to the reference dataset where the crypt-villi axis was calculated

In [2]:
xenium_path = 'D:/amonell/timecourse_final/day7_SI_DMSO'

In [3]:
reference = sc.read(os.path.join(xenium_path, 'adatas', '07_axes_defined_reference.h5ad'))

In [4]:
input_folders = glob.glob('D:/amonell/timecourse_final/day*')

### Calculate the epithelial axis for all datasets

In [5]:

sc.set_figure_params(dpi=1000, dpi_save=1000)
for input_file in input_folders:
    ad = sc.read(os.path.join(input_file, 'adatas', '06_before_decomposition_model.h5ad')) 
    points_epi = ad[ad.obs.Class.isin(['Epithelial'])].obsm['X_spatial']

    all_tree = KDTree(ad.obsm['X_spatial'])
    epi_tree = KDTree(points_epi)
    distances_all, neighbors_all = all_tree.query(ad.obsm['X_spatial'], k=5)
    distances, neighbors = epi_tree.query(ad.obsm['X_spatial'], k=5)
    distance_medians = (np.mean(distances, axis=1)/np.mean(distances_all, axis=1))
    ad.obs['epithelial_distance'] = distance_medians
    ad.obs['epithelial_distance'] = ad.obs['epithelial_distance']/np.percentile(ad.obs['epithelial_distance'], 99)
    fig = sc.pl.embedding(ad, basis = 'spatial', color='epithelial_distance', return_fig=True, show=False, vmax=1, cmap='viridis', size=4)
    fig.tight_layout()
    plt.axis('equal')
    fig.savefig(os.path.join(input_file, 'figures', 'axes', f'spatial_epithelial.png'))
    plt.close()
    ad.write(os.path.join(input_file, 'adatas', '06_before_decomposition_model.h5ad'))

C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):
C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and 

### Calculate the crypt-villi axis and normalize the longitudinal axis for all datasets

In [6]:
reference = reference[reference.obs['in_villi']]


C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [7]:
training_data = reference.obs[reference.obs.columns[reference.obs.columns.str.contains('Topic')]].values

In [8]:
training_labels = reference.obs['normalized_crypt_villi_scaled'].values

In [9]:
# Define model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(training_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(training_data, training_labels, epochs=10, batch_size=32, verbose= 2)

Epoch 1/10
6527/6527 - 8s - loss: 0.0224 - 8s/epoch - 1ms/step
Epoch 2/10
6527/6527 - 7s - loss: 0.0216 - 7s/epoch - 1ms/step
Epoch 3/10
6527/6527 - 7s - loss: 0.0214 - 7s/epoch - 1ms/step
Epoch 4/10
6527/6527 - 7s - loss: 0.0212 - 7s/epoch - 1ms/step
Epoch 5/10
6527/6527 - 7s - loss: 0.0211 - 7s/epoch - 1ms/step
Epoch 6/10
6527/6527 - 7s - loss: 0.0209 - 7s/epoch - 1ms/step
Epoch 7/10
6527/6527 - 7s - loss: 0.0208 - 7s/epoch - 1ms/step
Epoch 8/10
6527/6527 - 6s - loss: 0.0207 - 6s/epoch - 906us/step
Epoch 9/10
6527/6527 - 7s - loss: 0.0206 - 7s/epoch - 1ms/step
Epoch 10/10
6527/6527 - 6s - loss: 0.0205 - 6s/epoch - 963us/step


In [21]:
for input_file in input_folders:
    adata = sc.read(os.path.join(input_file, 'adatas', '06_before_decomposition_model.h5ad'))
    testing_data = adata.obs[adata.obs.columns[adata.obs.columns.str.contains('Topic')]].values
    predictions = model.predict(testing_data)
    adata.obs['crypt_villi_axis'] = predictions
    try:
        adata.obs['predicted_longitudinal'] = adata.obs['predicted_longitudinal'].replace(-1.0, np.nan)
        adata.obs['predicted_longitudinal'] = adata.obs['predicted_longitudinal']/100000
    except:
        adata.obs['predicted_longitudinal'] = adata.obs['longitudinal'].replace(-1.0, np.nan)
        adata.obs['predicted_longitudinal'] = adata.obs['predicted_longitudinal']/100000        
    
    adata.write(os.path.join(input_file, 'adatas', '07_axes_defined.h5ad'))
    fig = sc.pl.embedding(adata, basis = 'spatial', color='crypt_villi_axis', return_fig=True, show=False, vmax=1, cmap='viridis', size=4)
    fig.tight_layout()
    plt.axis('equal')
    fig.savefig(os.path.join(input_file, 'figures', 'axes', f'spatial_crypt_villi.png'))
    plt.close()

7912/7912 [==============================] - 10s 1ms/step


C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):


10184/10184 [==============================] - 10s 954us/step


C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):


7053/7053 [==============================] - 7s 940us/step


...

C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):


6956/6956 [==============================] - 7s 1ms/step


C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):


8063/8063 [==============================] - 7s 869us/step


C:\Users\amonell\anaconda3_2\envs\tensorflow\lib\site-packages\scanpy\plotting\_tools\scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):
